Dataset do Kaggle:

# Importações

In [166]:
import pandas as pd
import numpy as np
import math

In [167]:
data_movies = pd.read_csv('movies.csv')
data_ratings = pd.read_csv('ratings.csv')

In [168]:
data_ratings.head()

,Id,MovieId,Rating,Timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [169]:
data_movies.head()

,Id,Title,Genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# Varificando dados inconsistentes

Primeiro do dataset movie.csv

In [170]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      9742 non-null   int64 
 1   Title   9742 non-null   object
 2   Genres  9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [171]:
data_movies.isnull().sum()

Id        0
Title     0
Genres    0
dtype: int64

In [172]:
data_movies.isna().sum()

Id        0
Title     0
Genres    0
dtype: int64

In [173]:
data_movies.duplicated().sum()

0

Vamos separa o ano e coloca-la em uma coluna separada da coluna de título (Title).

In [174]:
data_movies['year'] = data_movies.Title.str.extract('(\(\d\d\d\d\))', expand=False)
data_movies['year'] = data_movies.year.str.extract('(\d\d\d\d)', expand=False)
data_movies['Title'] = data_movies.Title.str.replace('(\(\d\d\d\d\))', ' ')
data_movies['Title'] = data_movies['Title'].apply(lambda x: x.strip())
data_movies.head()

,Id,Title,Genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Agora iremos separa o gênero do filme em um outro Data Frame e retira-lo do formato de lista.

In [175]:
data_movies['Genres'] = data_movies['Genres'].str.split('|')
data_movies.head()

,Id,Title,Genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [176]:
df_genres = data_movies.copy()
for i, v in data_movies.iterrows():
  for c in v['Genres']:
    df_genres.at[i, c] = 1
df_genres.fillna(0, inplace=True)

In [177]:
df_genres.head()

,Id,Title,Genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Agora o dataset ratings.csv

In [178]:
data_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Id         100836 non-null  int64  
 1   MovieId    100836 non-null  int64  
 2   Rating     100836 non-null  float64
 3   Timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [179]:
data_ratings.isnull().sum()

Id           0
MovieId      0
Rating       0
Timestamp    0
dtype: int64

In [180]:
data_ratings.isna().sum()

Id           0
MovieId      0
Rating       0
Timestamp    0
dtype: int64

In [181]:
data_ratings.duplicated().sum()

0

Como não localizamos nenhum valor inconsistentes em nenhum dos dois datasets, iremos atribuir aos Data Frames definitivos. 

In [182]:
df_movies = df_genres
df_ratings = data_ratings

# Criando um usuário fictício

In [183]:
user = [
        {'Title':'Sabrina', 'Rating':2.5},
        {'Title':'Balto', 'Rating':5},
        {'Title':'Sense and Sensibility', 'Rating':3},
        {'Title':'Pocahontas', 'Rating':4.5},
        {'Title':'Vampire in Brooklyn', 'Rating':5}
]
user_movies = pd.DataFrame(user)
user_movies

,Title,Rating
0,Sabrina,2.5
1,Balto,5.0
2,Sense and Sensibility,3.0
3,Pocahontas,4.5
4,Vampire in Brooklyn,5.0


Aqui em cima estabelecemos um usuário hipotético para testar nosso algoritmo.

# Recomendação por gênero de filme

In [144]:
entrada = data_movies[data_movies['Title'].isin(user_movies['Title'].tolist())]
entrada_movies = pd.merge(entrada, user_movies)
entrada_movies = entrada_movies.drop('Genres', 1).drop('year', 1)
entrada_movies

,Id,Title,Rating
0,7,Sabrina,2.5
1,915,Sabrina,2.5
2,13,Balto,5.0
3,17,Sense and Sensibility,3.0
4,48,Pocahontas,4.5
5,93,Vampire in Brooklyn,5.0


In [145]:
user_movies = df_genres[df_genres['Id'].isin(entrada_movies['Id'].tolist())]
user_movies

,Id,Title,Genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
6,7,Sabrina,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,13,Balto,"[Adventure, Animation, Children]",1995,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,17,Sense and Sensibility,"[Drama, Romance]",1995,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,48,Pocahontas,"[Animation, Children, Drama, Musical, Romance]",1995,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
82,93,Vampire in Brooklyn,"[Comedy, Horror, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
697,915,Sabrina,"[Comedy, Romance]",1954,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [146]:
user_movies.reset_index(drop=True, inplace=True)
user_gender = user_movies.drop('Id', 1).drop('Title', 1).drop('Genres', 1).drop('year', 1)
user_gender

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Acima acabamos de selecionar somente os gêneros dos filmes assistidos pelo usuário.

In [147]:
user_weigth = user_gender.transpose().dot(entrada_movies['Rating'])
user_weigth

Adventure              2.5
Animation              5.5
Children               5.5
Comedy                12.0
Fantasy                0.0
Romance               20.0
Drama                  8.0
Action                 0.0
Crime                  0.0
Thriller               0.0
Horror                 4.5
Mystery                0.0
Sci-Fi                 0.0
War                    0.0
Musical                3.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Acima estabelecemos um peso para cada gênero envolvido nos filmes assistidos pelo cliente. Agora que temos o perfil do usuário vamos começar a recomendar outros filmes para ele.

In [148]:
genero_tabela = df_genres.set_index(df_genres['Id'])
genero_tabela = genero_tabela.drop('Id', 1).drop('Title', 1).drop('Genres', 1).drop('year', 1)
genero_tabela.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
Id,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Agora com o gênero de filmes em mãos vamos efetuar a média ponderada e selecionar os 20 filmes mais recomendados.

In [149]:
recomendacoes = ((genero_tabela * user_weigth).sum(axis=1)) / (user_weigth.sum())
recomendacoes = recomendacoes.sort_values(ascending=False)
recomendacoes.head()

Id
1907      0.926230
108540    0.877049
138702    0.836066
46062     0.795082
56152     0.795082
dtype: float64

In [150]:
df_movies.loc[df_movies['Id'].isin(recomendacoes.head(20).keys())]

,Id,Title,Genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
696,914,My Fair Lady,"[Comedy, Drama, Musical, Romance]",1964,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
812,1064,Aladdin and the King of Thieves,"[Animation, Children, Comedy, Fantasy, Musical...",1996,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1390,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1545,2081,"Little Mermaid, The","[Animation, Children, Comedy, Musical, Romance]",1989,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3194,4306,Shrek,"[Adventure, Animation, Children, Comedy, Fanta...",2001,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4278,6245,Sweet Charity,"[Comedy, Drama, Musical, Romance]",1969,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5160,8360,Shrek 2,"[Adventure, Animation, Children, Comedy, Music...",2004,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6230,46062,High School Musical,"[Children, Comedy, Drama, Musical, Romance]",2006,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6626,56152,Enchanted,"[Adventure, Animation, Children, Comedy, Fanta...",2007,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Acima chegamos a nossa tabela com as 20 recomendações.

# Recomendação por filtragem colaborativa

In [184]:
ent_id = df_movies[df_movies['Title'].isin(user_movies['Title'].tolist())]
ent_filme = pd.merge(ent_id, user_movies)
ent_filme = ent_filme.drop('year', 1)
ent_filme.head()

,Id,Title,Genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),Rating
0,7,Sabrina,"[Comedy, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5
1,915,Sabrina,"[Comedy, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5
2,13,Balto,"[Adventure, Animation, Children]",1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,17,Sense and Sensibility,"[Drama, Romance]",0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,48,Pocahontas,"[Animation, Children, Drama, Musical, Romance]",0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.5


Agora com o id's dos filmes podemos verificar quais os usuários que assistiram os mesmos filmes.

In [185]:
users_df = df_ratings[df_ratings['Id'].isin(ent_filme['Id'].tolist())]
users_df.head()

,Id,MovieId,Rating,Timestamp
874,7,1,4.5,1106635946
875,7,50,4.5,1106635993
876,7,58,3.0,1106635520
877,7,150,4.5,1106635925
878,7,165,4.0,1106635987


In [186]:
users_dfGroup = users_df.groupby(['Id'])

In [187]:
np.unique(users_df['Id'])

array([ 7, 13, 17, 48, 93])

In [188]:
users_dfGroup.get_group(93).head()

,Id,MovieId,Rating,Timestamp
14720,93,1,3.0,942767337
14721,93,2,5.0,942946208
14722,93,10,4.0,942946208
14723,93,15,4.0,942946091
14724,93,34,5.0,942767143


In [189]:
users_dfGroup = sorted(users_dfGroup, key=lambda x: len(x[1]), reverse=True)
users_dfGroup[0:3]

[(7,       Id  MovieId  Rating   Timestamp
  874    7        1     4.5  1106635946
  875    7       50     4.5  1106635993
  876    7       58     3.0  1106635520
  877    7      150     4.5  1106635925
  878    7      165     4.0  1106635987
  ...   ..      ...     ...         ...
  1021   7    48997     2.5  1174263740
  1022   7    49272     4.5  1165876367
  1023   7    49278     3.5  1174263775
  1024   7    49286     0.5  1176181731
  1025   7    49824     3.0  1173051469
  
  [152 rows x 4 columns]), (17,       Id  MovieId  Rating   Timestamp
  1667  17        1     4.5  1305696483
  1668  17       44     3.5  1305696245
  1669  17       47     4.0  1307262715
  1670  17       50     4.5  1305697013
  1671  17      110     4.5  1305696470
  ...   ..      ...     ...         ...
  1767  17    78499     4.0  1305696624
  1768  17    79132     4.5  1305696716
  1769  17    80463     4.0  1305696734
  1770  17    81932     3.5  1305696681
  1771  17    82459     4.0  1305696644
  
 

Classificamos os usuários que mais compartilham filmes em comum, para evitarmos analisar todos os usuários.

In [190]:
users_dfGroup = users_dfGroup[0:100]

In [191]:
pearson = dict()
for nome, valor in users_dfGroup:
  valor = valor.sort_values(by='Id')
  ent_filme = ent_filme.sort_values(by='Id')
  num_rat = len(valor)
  temp = ent_filme[ent_filme['Id'].isin(valor['Id'].tolist())]
  temp_rat = temp['Rating'].tolist()
  temp_group = valor['Rating'].tolist()
  x = sum([pow(i, 2) for i in temp_rat]) - pow(sum(temp_rat), 2) / float(num_rat)
  y = sum([pow(c, 2) for c in temp_group]) - pow(sum(temp_group), 2) / float(num_rat)
  xy = sum(i*j for i, j in zip(temp_rat, temp_group)) - sum(temp_rat) * sum(temp_group) / float(num_rat)
  if x != 0 and y != 0:
    pearson[nome] = (xy / math.sqrt(x * y)) 
  else:
    pearson[nome] = 0

In [192]:
pearson.items()

dict_items([(7, 0.07797216104386773), (17, 0.05628455977768995), (93, -0.22242133725536326), (48, -0.008233311524234814), (13, 0.25532032834562696)])

In [193]:
pearson_df = pd.DataFrame.from_dict(pearson, orient='index')
pearson_df.columns = ['similarindex']
pearson_df['Id'] = pearson_df.index
pearson_df.index = range(len(pearson_df))
pearson_df.head()

,similarindex,Id
0,0.077972,7
1,0.056285,17
2,-0.222421,93
3,-0.008233,48
4,0.255320,13


Agora vamos obter os 50 principais usuários que são mais semelhantes à entrada.

In [194]:
top = pearson_df.sort_values(by='similarindex', ascending=False)[0:50]
top.head()

,similarindex,Id
4,0.255320,13
0,0.077972,7
1,0.056285,17
3,-0.008233,48
2,-0.222421,93


Agora iremos começar a recomendar os filmes usando a correlação de pearson como peso e a média ponderada.

In [195]:
top_rating  = top.merge(df_ratings, left_on='Id', right_on='Id', how='inner')
top_rating.head()

,similarindex,Id,MovieId,Rating,Timestamp
0,0.25532,13,47,5.0,987895819
1,0.25532,13,305,1.0,987456968
2,0.25532,13,597,3.0,987456968
3,0.25532,13,1173,3.0,987457052
4,0.25532,13,1198,5.0,987456848


In [196]:
top_rating['aval_pond'] = top_rating['similarindex'] * top_rating['Rating']
temptop_rating = top_rating.groupby('Id').sum()[['similarindex', 'aval_pond']]
temptop_rating.columns = ['sum_similarindex', 'sum_aval_pond']
temptop_rating.head()

,sum_similarindex,sum_aval_pond
Id,,
7,11.851768,38.284331
13,7.914930,28.851197
17,5.909879,24.877775
48,-0.271699,-1.095030
93,-21.574870,-92.527276


In [197]:
recomendacao = pd.DataFrame()
recomendacao['peso_media_score'] = temptop_rating['sum_aval_pond'] / temptop_rating['sum_similarindex']
recomendacao['Id'] = recomendacao.index
recomendacao.head()

,peso_media_score,Id
Id,,
7,3.230263,7
13,3.645161,13
17,4.209524,17
48,4.030303,48
93,4.288660,93


In [198]:
recomendacao = recomendacao.sort_values(by='peso_media_score', ascending=False)
df_movies.loc[df_movies['Id'].isin(recomendacao.head(10)['Id'].tolist())]

,Id,Title,Genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
6,7,Sabrina,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,13,Balto,"[Adventure, Animation, Children]",1995,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,17,Sense and Sensibility,"[Drama, Romance]",1995,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,48,Pocahontas,"[Animation, Children, Drama, Musical, Romance]",1995,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
82,93,Vampire in Brooklyn,"[Comedy, Horror, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


E acima conseguimos as 5 recomendações dos próximos filmes.